In [42]:
# coding: utf-8
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
import pandas as pd
import gc
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import CoherenceModel, TfidfModel, LdaModel, LdaMulticore
from gensim.models.hdpmodel import HdpModel, HdpTopicFormatter
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric
from itertools import chain, islice
from constants import (
    FULL_PATH, ETL_PATH, NLP_PATH, SMPL_PATH, POS, NOUN, PROPN, TOKEN, HASH, SENT_IDX, PUNCT
)
import logging
import json
import numpy as np

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#pd.options.display.max_rows = 2001

In [2]:
def docs_to_lists(token_series):
    return tuple(token_series.tolist())

def docs2corpora(documents, tfidf=True, stopwords=None, filter_below=5, filter_above=0.5,
                split=False, max_test_size_rel=0.1, max_test_size_abs=5000):
    dictionary = Dictionary(documents)
    dictionary.filter_extremes(no_below=filter_below, no_above=filter_above)

    # filter some noice (e.g. special characters)
    if stopwords:
        stopword_ids = [dictionary.token2id[token] for token in stopwords]
        dictionary.filter_tokens(bad_ids=stopword_ids, good_ids=None)
        
    corpora = dict()
    if split:
        if length*max_test_size_rel < max_test_size_abs:
            split1 = int(length*(1-(2*max_test_size_rel)))
            split2 = int(length*(1-max_test_size_rel))
        else:
            split1 = length-(2*max_test_size_abs)
            split2 = length-max_test_size_abs
        training_texts = documents[:split1]
        holdout_texts = documents[split1:split2]
        test_texts = documents[split2:]
        corpora['training_corpus'] = [dictionary.doc2bow(text) for text in training_texts]
        corpora['holdout_corpus'] = [dictionary.doc2bow(text) for text in holdout_texts]
        corpora['test_corpus'] = [dictionary.doc2bow(text) for text in test_texts]
    else:
        training_texts = documents
        corpora['training_corpus'] = [dictionary.doc2bow(text) for text in training_texts]
        corpora['holdout_corpus'], corpora['test_corpus'] = None, None

    if tfidf:
        for key, bow_corpus in corpora.items():
            tfidf_model = TfidfModel(bow_corpus)
            corpora[key] = tfidf_model[bow_corpus]
    return corpora, dictionary

In [3]:
datasets = [
    'Europarl',
    'FAZ_combined',
    'FOCUS_cleansed',
    'OnlineParticipation',
    'PoliticalSpeeches',
    'dewiki',
    'dewac',
]
goodids = {
    # filetered via some fixed rules and similarity measure to character distribution
    'dewac': join(ETL_PATH, 'dewac_good_ids.pickle'),
    'dewiki': join(ETL_PATH, 'dewiki_good_ids.pickle'),
    # the samples contain only a small subset of all articles
    # the reason for this is that the samples are roughly equal in size per category
    # 'FAZ_combined': join(ETL_PATH, 'FAZ_document_sample3.pickle'),
    # 'FOCUS_cleansed': join(ETL_PATH, 'FOCUS_document_sample3.pickle'),
}
bad_tokens = {
    'Europarl': [
        'E.', 'Kerr', 'The', 'la', 'ia', 'For', 'Ieke', 'the',
    ],
    'FAZ_combined': [
        'S.', 'j.reinecke@faz.de', 'B.',
    ],
    'FOCUS_cleansed': [],
    'OnlineParticipation': [
        'Re', '@#1', '@#2', '@#3', '@#4', '@#5', '@#6', '@#7', '@#8', '@#9', '@#1.1', 'Für', 'Muss',
        'etc', 'sorry', 'Ggf', 'u.a.', 'z.B.'
        'B.', 'stimmt', ';-)', 'lieber', 'o.', 'Ja', 'Desweiteren',
    ],
    'PoliticalSpeeches': [],
    'dewiki': [],
    'dewac': [],
}
all_bad_tokens = set(chain(*bad_tokens.values()))

In [4]:
#for dataset in datasets[1:2]:
dataset = datasets[3]
print('dataset:', dataset)
dir_path = join(SMPL_PATH, 'wiki_phrases')
files = sorted([f for f in listdir(dir_path) if f.startswith(dataset)])
for name in files:
    full_path = join(dir_path, name)
    if isdir(full_path):
        subdir = sorted([join(name, f) for f in listdir(full_path) if f.startswith(dataset)])
        files += subdir

keepids = None
if dataset in goodids:
    keepids = pd.read_pickle(goodids[dataset])

documents = []
for name in files:
    gc.collect()
    full_path = join(dir_path, name)
    if not isfile(full_path):
        continue

    print('reading', name)
    df = pd.read_pickle(join(dir_path, name))
    print('initial number of words:', len(df))
    print('reducing number of tokens')
    if keepids is not None:
        # some datasets have already been filtered so you may not see a difference in any case
        df = df[df.hash.isin(keepids.index)]

    # fixing bad POS tagging
    mask = df.token.isin(['[', ']', '<', '>', '/', '–', '%'])
    df.loc[mask, POS] = PUNCT

    # using only certain POS tags
    df = df[df.POS.isin({NOUN, PROPN, 'NER', 'NPHRASE'})]
    df[TOKEN] = df[TOKEN].map(lambda x: x.strip('-/'))
    df = df[df.token.str.len() > 1]
    df = df[~df.token.isin(all_bad_tokens)]
    print('final number of words:', len(df))
    # groupby sorts the documents by hash-id which is equal to shuffeling the dataset before building the model
    df = df.groupby([HASH])[TOKEN].agg(docs_to_lists)
    documents += df.values.tolist()

length = len(documents)
print('number of documents:', length)
del keepids, files
gc.collect();

dataset: OnlineParticipation
reading OnlineParticipation_simple_wiki_phrases.pickle
initial number of words: 1661338
reducing number of tokens
final number of words: 375942
number of documents: 25663


In [16]:
corpora, dictionary = docs2corpora(
    documents, tfidf=False,
    #stopwords=bad_tokens[dataset],  # stopword removale has been moved to the pandas preprocessing pipeline
    filter_below=5, filter_above=0.5,
    split=True,
)
training_corpus = corpora['training_corpus']
holdout_corpus = corpora['holdout_corpus']
test_corpus = corpora['test_corpus']

def init_callbacks(viz_env=None, title_suffix=''):
    # define perplexity callback for hold_out and test corpus
    pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="visdom", viz_env=viz_env, title="Perplexity (hold_out)"+title_suffix)
    pl_test = PerplexityMetric(corpus=test_corpus, logger="visdom", viz_env=viz_env, title="Perplexity (test)"+title_suffix)

    # define other remaining metrics available
    ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", topn=10, logger="visdom", viz_env=viz_env, title="Coherence (u_mass)"+title_suffix)
    ch_cv = CoherenceMetric(corpus=training_corpus, texts=documents, coherence="c_v", topn=10, logger="visdom", viz_env=viz_env, title="Coherence (c_v)"+title_suffix)
    diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", viz_env=viz_env, title="Diff (kullback_leibler)"+title_suffix)
    convergence_kl = ConvergenceMetric(distance="jaccard", logger="visdom", viz_env=viz_env, title="Convergence (jaccard)"+title_suffix)

    return [pl_holdout, pl_test, ch_umass, ch_cv, diff_kl, convergence_kl]

In [6]:
def get_parameterset(corpus, dictionary, callbacks=None, nbtopics=100, parametrization='a42', eval_every=None):
    print(f'building LDA model "{parametrization}" with {nbtopics} number of topics')
    default = dict(random_state=42, corpus=corpus, id2word=dictionary, num_topics=nbtopics, 
                   eval_every=eval_every, callbacks=callbacks, chunksize=20_000)
    ldamodels = {
        'a42': dict(),
        'b42': dict(passes=10, iterations=100),
        'c42': dict(passes=10, iterations=10_000),
        'd42': dict(passes=10, iterations=200, alpha=0.1, eta=0.01),
        'x42': dict(passes=20, iterations=1000, alpha='auto', eta='auto'),
    }
    for key, dic in ldamodels.items():
        dic.update(default)
    return ldamodels[parametrization]

Install Visdom via ```pip install visdom``` and run ```python -m visdom.server``` to start the server.
You will be able to view online log stats @ http://localhost:8097/

Attention: gensim is currently not fully compatible with visdom. See https://github.com/RaRe-Technologies/gensim/issues/2155 for details.

To fix this issue change in gensim.models.callbacks.Callback.on_epoch_end the following line
```
self.viz.updateTrace(
    Y=np.array([value]), X=np.array([epoch]), env=metric.viz_env, win=self.windows[i]
)
```
to
```
self.viz.line(
    Y=np.array([value]), X=np.array([epoch]), env=metric.viz_env, win=self.windows[i], update='append'
)
```
You might need to re-import gensim (and probably even restart this notebook).

In [46]:
params = ['x42', 'a42', 'b42', 'c42', 'd42'][-1]
implementations = [
    ('LDAmodel', LdaModel),
    ('LDAmulticore', LdaMulticore)
]
choice = 0
model_name = implementations[choice][0]
UsedModel = implementations[choice][1]
save = True
metrics = []
env_id = f"{dataset}-{model_name}-{params}"
for nbtopics in range(10, 101, 10):
    # Choose α from [0.05, 0.1, 0.5, 1, 5, 10]
    # Choose β from [0.05, 0.1, 0.5, 1, 5, 10]
    callbacks = init_callbacks(viz_env=env_id, title_suffix=f" | {nbtopics} topics")
    kwargs = get_parameterset(training_corpus, dictionary, callbacks=callbacks, nbtopics=nbtopics, parametrization=params)
    if 'multicore' in model_name:
        kwargs['workers'] = 3
    ldamodel = UsedModel(**kwargs)

    topics = [[dataset] + [dictionary[term[0]] for term in ldamodel.get_topic_terms(i)] for i in range(nbtopics)]
    df_lda = pd.DataFrame(topics, columns=['dataset']+['term'+str(i) for i in range(10)])

    # calculate (average) UMass score
    #top_topics = ldamodel.top_topics(training_corpus, topn=10, processes=8)
    # Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
    #avg_topic_coherence = sum([t[1] for t in top_topics]) / nbtopics
    #print('Average topic coherence: %.4f.' % avg_topic_coherence)
    current_metrics = ldamodel.metrics
    print(current_metrics)
    metrics.append(('env_id', current_metrics))

    if save:
        out_dir = join(ETL_PATH, f'{model_name}/{params}')
        if not exists(out_dir):
            makedirs(out_dir)
        out = join(out_dir, f'{dataset}_{model_name}_{params}_{nbtopics}')
        print('saving to', out)
        df_lda.to_csv(out + '.csv')
        ldamodel.save(out)
        with open(out + '_metrics.json', 'w') as fp:
            serializable_metrics = {}
            for k, v in current_metrics.items():
                if isinstance(v[0], np.ndarray):
                    serializable_metrics[k] = [x.tolist() for x in v]
                else:
                    serializable_metrics[k] = [float(x) for x in v]
            json.dump(serializable_metrics, fp)

building LDA model "d42" with 10 number of topics


/home/andreas/bin/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


defaultdict(<class 'list'>, {'Perplexity (hold_out) | 10 topics': [5096.9190789029, 2928.7997410063567, 1751.257394789178, 1231.9200346328369, 987.8664996848179, 857.2400337059956, 778.9205036519083, 728.4404881567187, 693.6528240087434, 669.0051764297576], 'Perplexity (test) | 10 topics': [5312.882484689098, 3033.548406189043, 1808.9170214194328, 1268.9769344156712, 1015.0047524540354, 879.4458826871764, 798.713355877408, 746.8234489849148, 711.5117711492551, 685.7000600154025], 'Coherence (u_mass) | 10 topics': [-2.7548603418888424, -2.652116886541265, -2.9918325587118964, -2.7308262359958446, -2.7406916212486374, -2.6045957351156916, -2.605917692473647, -2.5934489220157437, -2.5021117559987, -2.485888933834015], 'Coherence (c_v) | 10 topics': [0.4773905354360496, 0.5382716771487651, 0.5753546775332874, 0.6017238526246877, 0.5939963184718172, 0.6009822826596427, 0.6042310663132253, 0.6149887880543139, 0.6201653463698727, 0.6272596816238327], 'Diff (kullback_leibler) | 10 topics': [ar

-----

In [ ]:
top_topics = ldamodel.top_topics(corpus, topn=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / len(top_topics)
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

In [ ]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary, random_state=42)

In [ ]:
htf = HdpTopicFormatter(dictionary, topic_data=hdpmodel.get_topics())
topics = [[dataset] + [term[0] for term in htf.show_topic(i, topn=10, formatted=False)] for i in range(nbtopics)]
df_hdp = pd.DataFrame(topics, columns=['dataset']+['term'+str(i) for i in range(10)])
df_hdp

In [ ]:
ldamodel.metrics

In [ ]:
def topic_prob_extractor(gensim_hdp):
    shown_topics = gensim_hdp.show_topics(num_topics=-1, formatted=False)
    topics_nos = [x[0] for x in shown_topics ]
    weights = [ sum([item[1] for item in shown_topics[topicN][1]]) for topicN in topics_nos ]

    return pd.DataFrame({'topic_id' : topics_nos, 'weight' : weights})

topic_prob_extractor(hdpmodel)

In [ ]:
topics = []
for topic_id, topic in hdpmodel.show_topics(num_topics=10, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)
topics[:2]

In [ ]:
coherences = []
for topic in topics:
    cm = CoherenceModel(topics=[topic], corpus=corpus, dictionary=dictionary, coherence='u_mass')
    coherences.append(cm.get_coherence())
coherences

In [ ]:
sum(coherences) / len(coherences)

In [ ]:
cmlda = CoherenceModel(model=ldamodel, corpus=corpus, coherence='u_mass')
cmlda.get_coherence()
cmhdp = CoherenceModel(model=hdpmodel, corpus=corpus, coherence='u_mass')
cmhdp.get_coherence()

In [ ]:
cm = CoherenceModel.for_topics(topics_as_topn_terms=topics, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [ ]:
cm.get_coherence_per_topic(segmented_topics=topics, with_std=True, with_support=True)


### Evaluation

In [ ]:
MmCorpus.serialize('../data/{}.mm'.format(dataset), corpus)
corpus_fake = MmCorpus('../data/{}.mm'.format(dataset))
prepared_data = ldavis.gensim.prepare(ldamodel, corpus_fake, dictionary)
prepared_data

In [ ]:
# To get the topic words from the model
topics = []
for topic_id, topic in hm.show_topics(num_topics=10, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)
topics[:2]

# Initialize CoherenceModel using `topics` parameter
coherences = []
for topic in topics:
    cm = CoherenceModel(topics=[topic], corpus=corpus, dictionary=dictionary, coherence='u_mass')
    coherences.append(cm.get_coherence())
coherences

sum(coherences) / len(coherences)

### Start c_v coherence measure
This is expected to take much more time since `c_v` uses a sliding window to perform probability estimation and uses the cosine similarity indirect confirmation measure.

In [ ]:
%%time

cm = CoherenceModel(topics=usable_topics, texts=texts, dictionary=dictionary, coherence='c_v')
c_v = cm.get_coherence_per_topic()
print("Calculated c_v coherence for %d topics" % len(c_v))

### Start c_uci and c_npmi coherence measures
c_v and c_uci and c_npmi all use the boolean sliding window approach of estimating probabilities. Since the `CoherenceModel` caches the accumulated statistics, calculation of c_uci and c_npmi are practically free after calculating c_v coherence. These two methods are simpler and were shown to correlate less with human judgements than c_v but more so than u_mass.

In [ ]:
%%time

cm.coherence = 'c_uci'
c_uci = cm.get_coherence_per_topic()
print("Calculated c_uci coherence for %d topics" % len(c_uci))

In [ ]:
%%time

cm.coherence = 'c_npmi'
c_npmi = cm.get_coherence_per_topic()
print("Calculated c_npmi coherence for %d topics" % len(c_npmi))

In [ ]:
final_scores = [
    score for i, score in enumerate(human_scores)
    if i not in invalid_topic_indices
]
len(final_scores)

The [values in the paper](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) were:

__`u_mass` correlation__ : 0.093

__`c_v` correlation__    : 0.548

__`c_uci` correlation__  : 0.473

__`c_npmi` correlation__ : 0.438

Our values are also very similar to these values which is good. This validates the correctness of our pipeline, as we can reasonably attribute the differences to differences in preprocessing.

In [ ]:
for our_scores in (u_mass, c_v, c_uci, c_npmi):
    print(pearsonr(our_scores, final_scores)[0])

In [ ]:
sum(u_mass)/99, sum(c_v)/99, sum(c_uci)/99, sum(c_npmi)/99

In [ ]:
cm.save('movies_coherence_model')

In [ ]:
from gensim import utils


def report_on_oov_terms(cm, topic_models):
    """OOV = out-of-vocabulary"""
    topics_as_topn_terms = [
        models.CoherenceModel.top_topics_as_word_lists(model, dictionary)
        for model in topic_models
    ]

    oov_words = cm._accumulator.not_in_vocab(topics_as_topn_terms)
    print('number of oov words: %d' % len(oov_words))
    
    for num_topics, words in zip(trained_models.keys(), topics_as_topn_terms):
        oov_words = cm._accumulator.not_in_vocab(words)
        print('number of oov words for num_topics=%d: %d' % (num_topics, len(oov_words)))

report_on_oov_terms(cm, trained_models.values())